In [1]:
import os
import sys
import json
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# sys.path.append('/Users/pwong/Documents/AC297r_2019_Associated_Press/env/lib/python3.6/site-package')
# sys.path.append('/Users/pwong/anaconda3/bin')

import textacy
import pandas as pd
import numpy as np

In [2]:
from src.nlp_util.textacy_util import *

In [3]:
%load_ext autoreload
%autoreload 2

In [58]:
art_sub = pd.read_csv("../data/clean_data/article_subject.csv")
art_person = pd.read_csv("../data/clean_data/article_person.csv")
art_place = pd.read_csv("../data/clean_data/article_place.csv")
art_org = pd.read_csv("../data/clean_data/article_org.csv")

In [72]:
art_sub['type'] = 'subject'
art_person['type'] = 'person'
art_place['type'] = 'place'
art_org['type'] = 'org'

art_sub = art_sub.rename(columns={"subject_tag": "tag", "subject_tag_relation": "tag_relation", "subject_tag_code": "tag_code"})
art_person = art_person.rename(columns={"person_tag": "tag", "person_tag_relation": "tag_relation", "person_tag_code": "tag_code"})
art_place = art_place.rename(columns={"place_tag": "tag", "place_tag_relation": "tag_relation", "place_tag_code": "tag_code"})
art_org = art_org.rename(columns={"org_tag": "tag", "org_tag_relation": "tag_relation", "org_tag_code": "tag_code"})

art_alltags = pd.concat([art_sub, art_person, art_place, art_org])

In [73]:
art_alltags.head()

,id,tag,tag_relation,tag_code,type
0,964285bee74e430cb22441e59a968e76,General news,direct_ancestor,f25af2d07e4e100484f5df092526b43e,subject
1,964285bee74e430cb22441e59a968e76,Crime,direct,48612b807c2b4b93bbbacbf54f547ad5,subject
2,964285bee74e430cb22441e59a968e76,Arrests,direct,b2a4c0a08315100485d9d5a57deba703,subject
3,964285bee74e430cb22441e59a968e76,Law and order,ancestor,17956128897810048ce8d56c852d093e,subject
4,964285bee74e430cb22441e59a968e76,Criminal investigations,direct,d1720f70883210048576ae2ac3a6923e,subject


In [77]:
art_alltags[art_alltags['id'] == article_id][['tag', 'type']]

,tag,type
18428,General news,subject
18429,Government and politics,subject
18430,Funerals and memorial services,subject
3708,Recep Tayyip Erdogan,person
3709,Kemal Kilicdaroglu,person
6860,Ankara,place
6861,Turkey,place
6862,Western Europe,place
6863,Europe,place
6864,Middle East,place


In [53]:
with open("../data/full_text/0a0e0db8ae42425897b6381481663611.json") as json_file:
    data = json.load(json_file)
article_id = data['id']
text = data['full_text']

In [5]:
data

{'id': '0a0e0db8ae42425897b6381481663611',
 'full_text': 'ANKARA, Turkey (AP) — Turkey\'s interior minister says nine people have been detained in the assault of an opposition party leader, who was hit during a soldier\'s funeral.Several protesters threw punches at Republican People\'s Party leader Kemal Kilicdaroglu at the funeral outside Ankara on Sunday. Kilicdaroglu was not injured.Interior Minister Suleyman Soylu said Monday nine people were detained for questioning.The soldier was killed Saturday in clashes with Kurdish rebels. Soylu appeared to justify Kilicdaroglu\'s assault by referring to the support a pro-Kurdish gave the opposition during Turkey\'s March 31 municipal elections.Soylu said: "Everyone must take sides against the (rebels)."The Republican People\'s Party won the mayoral elections in Ankara and Istanbul, supplanting President Recep Tayyip Erdogan\'s party.Erdogan led a divisive campaign, equating opposition parties with terrorists.'}

In [9]:
# with open("../data/article/0a0e0db8ae42425897b6381481663611.json") as json_file:
#     metadata = json.load(json_file)
# # text = data['full_text']
# for people in metadata['data']['item']['person']:
#     print(people['name'])

# # Get tagging API entities
# entities = []
# cat_type = ['person', 'subject', 'organisation', 'place']
# for cat in cat_type:
#     for data in metadata['data']['item'][cat]:
#         entities.append(data['name']) # concat all metadata into this
# entities = np.array(entities)
# entities = entities[np.where(entities != "i")] # drop "i" from editor

In [87]:
text

'ANKARA, Turkey (AP) — Turkey\'s interior minister says nine people have been detained in the assault of an opposition party leader, who was hit during a soldier\'s funeral.Several protesters threw punches at Republican People\'s Party leader Kemal Kilicdaroglu at the funeral outside Ankara on Sunday. Kilicdaroglu was not injured.Interior Minister Suleyman Soylu said Monday nine people were detained for questioning.The soldier was killed Saturday in clashes with Kurdish rebels. Soylu appeared to justify Kilicdaroglu\'s assault by referring to the support a pro-Kurdish gave the opposition during Turkey\'s March 31 municipal elections.Soylu said: "Everyone must take sides against the (rebels)."The Republican People\'s Party won the mayoral elections in Ankara and Istanbul, supplanting President Recep Tayyip Erdogan\'s party.Erdogan led a divisive campaign, equating opposition parties with terrorists.'

In [6]:
en = textacy.load_spacy_lang("en_core_web_sm", disable=("parser",))
doc = textacy.make_spacy_doc(text, lang=en)

In [92]:
import textacy.ke

In [95]:
textacy.ke.yake(doc)

ValueError: [E030] Sentence boundaries unset. You can add the 'sentencizer' component to the pipeline with: nlp.add_pipe(nlp.create_pipe('sentencizer')) Alternatively, add the dependency parser, or set sentence boundaries by setting doc[i].is_sent_start.

In [89]:
# Extract textrank of named entities
textrank_entities, textrank_score, entities_list = extract_textrank_from_text(doc, tagging_API_entities = entities)
textrank_entities, textrank_score, entities_list

Textrank words ('Party leader Kemal Kilicdaroglu', 'opposition party leader', 'Interior Minister Suleyman Soylu', 'President Recep Tayyip Erdogan', 'municipal election', 'kurdish rebel', 'mayoral election', 'Republican People', 'Turkey', 'divisive campaign', 'interior minister', 'Ankara', 'assault', 'funeral', 'soldier', 'people', 'protester', 'punch', 'AP', 'Istanbul', 'March', 'Monday', 'Saturday', 'clash', 'support', 'Sunday', 'terrorist')
entities included: Kemal Kilicdaroglu
entities included: Recep Tayyip Erdogan
entities included: Turkey
entities included: Ankara


(array(['Party leader Kemal Kilicdaroglu',
        'President Recep Tayyip Erdogan', 'Turkey', 'Ankara'], dtype='<U32'),
 array([0.04750027, 0.0352918 , 0.01804562, 0.01335449]),
 array(['Ankara', 'Kemal Kilicdaroglu', 'Recep Tayyip Erdogan', 'Turkey'],
       dtype='<U30'))

In [96]:
pd.DataFrame([textrank_entities, textrank_score, entities_list]).T.rename(columns = \
              {0:"TextRank Bags of Words", 1:"TextRank Importance Score", 2:"Tagging API Named Entities"})

,TextRank Bags of Words,TextRank Importance Score,Tagging API Named Entities
0,Party leader Kemal Kilicdaroglu,0.0475003,Ankara
1,President Recep Tayyip Erdogan,0.0352918,Kemal Kilicdaroglu
2,Turkey,0.0180456,Recep Tayyip Erdogan
3,Ankara,0.0133545,Turkey


In [10]:
ts.basic_counts['n_words'] ## CHECK ## doesn't match

137

## TextRank

In [11]:
import textacy.ke

In [12]:
textacy.spacier.utils.get_normalized_text()

TypeError: get_normalized_text() missing 1 required positional argument: 'span_or_token'

In [83]:
textacy.TextStats(doc).basic_counts['n_words']


137

In [85]:
textacy.ke.textrank(doc, normalize="lemma", topn=10, window_size=3)

[('Party leader Kemal Kilicdaroglu', 0.047500269373415754),
 ('opposition party leader', 0.04491151046482916),
 ('Interior Minister Suleyman Soylu', 0.0393905668124247),
 ('President Recep Tayyip Erdogan', 0.03529180424934934),
 ('municipal election', 0.02023076940656441),
 ('kurdish rebel', 0.02017311337306259),
 ('mayoral election', 0.019878721600533583),
 ('Republican People', 0.019007513261570123),
 ('Turkey', 0.018045623982279352),
 ('divisive campaign', 0.014757984913261336)]

In [87]:
textacy.ke.textrank(doc, normalize="lemma", topn=10, window_size=2)

[('Party leader Kemal Kilicdaroglu', 0.04617915923493072),
 ('opposition party leader', 0.04196247513260644),
 ('President Recep Tayyip Erdogan', 0.03830877804676011),
 ('Interior Minister Suleyman Soylu', 0.037198656876395696),
 ('municipal election', 0.020586955135560664),
 ('kurdish rebel', 0.020301421625340613),
 ('mayoral election', 0.01961960693434969),
 ('Republican People', 0.017306950928731746),
 ('Turkey', 0.015486631465300623),
 ('divisive campaign', 0.015011340305379331)]

## Use named entities only in TextRank

In [31]:
entities_list_fulltext_df = pd.read_csv("../data/entities_list_full_text_dedup.csv")

In [32]:
entities_list_fulltext_df.shape, entities_list_fulltext_df.drop_duplicates(['text', 'article_id']).shape

((764798, 5), (764798, 5))

In [33]:
trial_id = '0a0e0db8ae42425897b6381481663611'
trial_entities_df = entities_list_fulltext_df[entities_list_fulltext_df['article_id'] == '0a0e0db8ae42425897b6381481663611']


In [34]:
trial_entities_df.shape, trial_entities_df.drop_duplicates(['text']).shape

((15, 5), (15, 5))

In [35]:
trial_entities_df

,Unnamed: 0,Unnamed: 0.1,text,label,article_id
50077,82862,0,ANKARA,ORG,0a0e0db8ae42425897b6381481663611
50078,82863,1,Turkey,GPE,0a0e0db8ae42425897b6381481663611
50079,82864,2,AP,ORG,0a0e0db8ae42425897b6381481663611
50080,82866,4,Republican People's Party,ORG,0a0e0db8ae42425897b6381481663611
50081,82867,5,Kemal Kilicdaroglu,PERSON,0a0e0db8ae42425897b6381481663611
50082,82868,6,Ankara,GPE,0a0e0db8ae42425897b6381481663611
50083,82869,7,Kilicdaroglu,PERSON,0a0e0db8ae42425897b6381481663611
50084,82870,8,Suleyman Soylu,PERSON,0a0e0db8ae42425897b6381481663611
50085,82871,9,Kurdish,NORP,0a0e0db8ae42425897b6381481663611
50086,82872,10,Soylu,ORG,0a0e0db8ae42425897b6381481663611


In [ ]:
extract_textrank_from_text()

In [50]:
# Get relevant entities
rel_gp = ['PERSON', 'GPE']
trial_entities = trial_entities_df['text'][trial_entities_df['label'].isin(rel_gp)].values
trial_entities

array(['Turkey', 'Kemal Kilicdaroglu', 'Ankara', 'Kilicdaroglu',
       'Suleyman Soylu', 'Istanbul', "Recep Tayyip Erdogan's", 'Erdogan'],
      dtype=object)

In [102]:
np.unique(trial_entities)

array(['Ankara', 'Erdogan', 'Istanbul', 'Kemal Kilicdaroglu',
       'Kilicdaroglu', "Recep Tayyip Erdogan's", 'Suleyman Soylu',
       'Turkey'], dtype=object)

In [78]:
doc

ANKARA, Turkey (AP) — Turkey's interior minister says nine people have been detained in the assault of an opposition party leader, who was hit during a soldier's funeral.Several protesters threw punches at Republican People's Party leader Kemal Kilicdaroglu at the funeral outside Ankara on Sunday. Kilicdaroglu was not injured.Interior Minister Suleyman Soylu said Monday nine people were detained for questioning.The soldier was killed Saturday in clashes with Kurdish rebels. Soylu appeared to justify Kilicdaroglu's assault by referring to the support a pro-Kurdish gave the opposition during Turkey's March 31 municipal elections.Soylu said: "Everyone must take sides against the (rebels)."The Republican People's Party won the mayoral elections in Ankara and Istanbul, supplanting President Recep Tayyip Erdogan's party.Erdogan led a divisive campaign, equating opposition parties with terrorists.

In [37]:
window_size = 2
trial_textrank = textacy.ke.textrank(doc, normalize="lemma", topn=10, window_size=window_size)
textrank_words = [textrank[0] for textrank in trial_textrank]

In [42]:
textrank_words

['Party leader Kemal Kilicdaroglu',
 'opposition party leader',
 'President Recep Tayyip Erdogan',
 'Interior Minister Suleyman Soylu',
 'municipal election',
 'kurdish rebel',
 'mayoral election',
 'Republican People',
 'Turkey',
 'divisive campaign']

In [101]:
get_textrank_entities_only(np.array(textrank_words), trial_entities)

array(['Party leader Kemal Kilicdaroglu',
       'President Recep Tayyip Erdogan',
       'Interior Minister Suleyman Soylu', 'Turkey'], dtype='<U32')